In [ ]:
from postprocess_base import read_logs
import altair as alt
from glob import glob

runs = read_logs(glob('../bps.log')) # other benchmarks can be found in 'noether-64/petsc-bps-2020-*.txt'
runs['FE_nodes_per_compute_node'] = runs['num_unknowns'] / (runs['num_procs'] / runs['num_procs_node']) / runs['dof_per_node']
runs.head()

In [ ]:
highlight = alt.selection_single(
    on='mouseover',
    fields=['degree', 'time_per_it', 'backend'],
    nearest=True,
    empty='none',
)

bps_select = alt.selection_single(
    fields=['bp'],
)

base = alt.Chart(runs).encode(
    alt.Y('mdofs:Q', title='GDoF/s per CG iteration'),
    alt.Color('degree:N'),
    alt.Size('num_unknowns', scale=alt.Scale(type='log', domain=(1e3, 1e6))),
    alt.Shape('bp:N'),
    tooltip=('bp', 'num_procs', 'backend', 'num_elem', 'degree', 'num_unknowns'),
).transform_filter(
    bps_select,
).transform_calculate(
    mdofs='datum.cg_iteration_dps/1e9',
).transform_filter(
    alt.FieldEqualPredicate(field='bp', equal=3)
)

points = base.mark_point().encode(
    opacity=alt.condition(highlight, alt.value(1), alt.value(.5))
).encode(size=alt.value(60)
).add_selection(
    highlight,
)

lines = base.mark_line().encode(
    size=alt.condition(alt.datum.degree - highlight.degree == 0, alt.value(3), alt.value(2))
)

pane = points + lines

composite = (
    pane.encode(
        alt.X('time_per_it', title = 'time per iteration [seconds]', scale=alt.Scale(type='log')),
    ) |
    pane.encode(
        alt.X('FE_nodes_per_compute_node', title = 'FE points per compute node', scale=alt.Scale(type='log', domain=(3e4, 1e7), clamp=True)),
    )
)

activator = alt.Chart(runs).mark_point().encode(
    alt.Y('bp', title='BP'),
    alt.Shape('bp')
).add_selection(bps_select).properties(title='Selection')

activator | composite.properties(title='Noether (2x EPYC 7452), gcc-10')

# If you want to save the plot
# plot =  activator | composite.properties(title='Noether (2x EPYC 7452), gcc-10')
# save(plot, 'BP4_with_title.pdf')